In [41]:
import deepquantum as dq
import deepquantum.photonic as dqp
import torch
import torch.nn as nn
import numpy as np

\begin{equation}
    \mathbb{S}_{\mathrm{BS}}=\begin{pmatrix}r&t\\-t&r\end{pmatrix},
\end{equation}

\begin{equation}
    r=\sin(\eta),~\theta=\eta-\pi/2,~\phi=0,
\end{equation}

\begin{equation}
    U_{\mathrm{BS}}(\theta,\phi)=\begin{pmatrix}\cos\left(\theta\right)&-e^{-i\phi}\sin\left(\theta\right)\\e^{i\phi}\sin\left(\theta\right)&\cos\left(\theta\right)\end{pmatrix}
\end{equation}

In [42]:
torch.randn(3, 3)

tensor([[ 0.0636, -1.9351,  0.4839],
        [-0.2981, -0.9414, -0.1829],
        [-0.2813,  0.1991, -0.0491]])

In [43]:
num_ME = 7
eta = 1
theta = eta - torch.pi/2

random_cov = torch.abs(torch.randn(2*(num_ME+1), 2*(num_ME+1)))
random_mean = torch.abs(torch.randn(2*(num_ME+1), 2*(num_ME+1)))

cov = random_cov
mean = random_mean

cov = torch.eye(2*(num_ME+1))
mean = torch.zeros(2*(num_ME+1))

# cir = dq.QumodeCircuit(nmode=num_ME+1, init_state=[cov, mean], 
#                        backend='gaussian', 
#                        name='try', noise=False)

cir = dq.QumodeCircuit(nmode=num_ME+1, init_state='vac', 
                       backend='gaussian', 
                       name='try', noise=False)

for i in range(num_ME-1):
    for j in range(num_ME-i-1):
        cir.bs(wires=[j+1,j+2], inputs=[theta, 0])
        cir.ps(j+1, [0.1])

#线路可视化
cir.draw('pic/CM_circ_num_ME.svg'.format(num_ME))

In [44]:
state = cir.forward()

# 这里measure_homodyne测量对应的物理量是正交算符 $$\hat x$$ 和 $$\hat p$$ 的值，
# photon_number_mean_var对应的是每个mode的光子数的平均值和方差。

# 第一个光子：x1，p1，x2，p2...
# 第二个光子：x1，p1，x2，p2...
sample = cir.measure_homodyne(shots=1000)

# ( [平均值], [方差] )
photon_number = cir.photon_number_mean_var()
print('number of modes: ', num_ME+1)
print('sample: ', sample)
print('size of sample: ', sample.size())
print('\n全部qubit的每光子数的平均值和方差: ', photon_number)

number of modes:  8
sample:  tensor([[-0.1802, -0.5746, -1.2918,  ...,  0.7609,  0.7123,  0.2027],
        [ 1.7162,  0.0857,  0.7725,  ...,  0.2982, -0.1655, -0.7667],
        [-1.8229, -0.7261,  1.5416,  ...,  0.9275,  0.0048, -0.3221],
        ...,
        [-0.1690, -0.1834, -0.8032,  ...,  0.2020, -0.9784,  1.1719],
        [ 0.2178,  2.0615,  0.7494,  ...,  0.9386,  1.0888, -0.6133],
        [-0.3305,  1.1871, -1.3987,  ..., -1.3671, -0.6377,  0.8692]])
size of sample:  torch.Size([1000, 16])

全部qubit的每光子数的平均值和方差:  (tensor([0.0000e+00, 1.7881e-07, 1.1921e-07, 1.1921e-07, 1.7881e-07, 1.7881e-07,
        5.9605e-08, 0.0000e+00]), tensor([0.0000e+00, 1.7881e-07, 1.1921e-07, 1.1921e-07, 1.7881e-07, 1.7881e-07,
        5.9605e-08, 0.0000e+00]))
